### Raw Data Processing

In [20]:
# import libraries

import os
import io
import re
import csv
from itertools import chain
from string import punctuation
from time import time
from nltk.tokenize import sent_tokenize, word_tokenize

In [24]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/lea/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

#### Helper Functions

In [2]:
def process_ann(ann_file):
    """Helper function that reads a .ann file,
       strips out newline characters, splits the tab-delimited entries,
       and extracts information for labeling corresponding .txt file
       
       Input:
       ann_file = tab-delimited brat annotation file with the following format
                  NER: [entity_ID]\t[label start_offset end_offset]\t[entity]
                  RE:  [relation_ID]\t[relation_type argument1 argument2]
       
       Outputs:
       cleaned_offsets = list of tuples for labeling corresponding .txt file
                         format: (offset, label, entity ID)
       corrections = dictionary of entity ID mappings for overlapping offsets"""
    
    with io.open(ann_file, 'r', encoding='utf-8', errors='ignore') as text:
        ann = [x.strip().split('\t') for x in text.readlines() if x.strip().split('\t')[0][0] == 'T']
    
    offsets = []
    
    for x in ann:
        entity_id = x[0]
        start = int(x[1].split()[1])
        end = int(x[1].split()[2])
        label = x[1].split()[0]
        
        offsets.append((start, 'S', label, entity_id))
        offsets.append((end, 'E', label, entity_id))
    
    sorted_offsets = sorted(offsets, key=lambda x:x[0])
    
    cleaned_offsets = []
    corrections = {}
    
    hold = None
    indicator = None
    
    for tup in sorted_offsets:
        
        if indicator == 'S':
            if tup[1] == 'E':
                cleaned_offsets.append(hold)
                hold = (tup[0], 'O', 'X')
                indicator = tup[1]
            elif tup[1] == 'S':
                corrections.update({tup[3]:hold[2]})
                indicator = '*'
        
        elif indicator == 'E':
            cleaned_offsets.append(hold)
            hold = (tup[0], tup[2], tup[3])
            indicator = tup[1]
        
        elif indicator == '*':
            indicator = 'S'

        else:
            hold = (tup[0], tup[2], tup[3])
            indicator = tup[1]
            
    cleaned_offsets.append(hold)
    
    return cleaned_offsets, corrections

In [60]:
def ann_chunker(txt_file, offsets):
    """Helper function that reads in a .txt file as one string,
       divides it based on the cleaned offsets from its .ann file
       and labels chunks with NER tags
       
       Inputs:
       txt_file = file that contains all the patent text
                  considered as one sentence in this task
       offsets = list of tuples for labeling corresponding .txt file
                 format: (offset, label, entity ID)
       
       Output:
       ann_chunks = list of annotated chunks based on .ann file offsets
                    format: (chunk, label)"""
    
    with io.open(txt_file, 'r', encoding='utf-8', errors='ignore') as text:
        full_text = text.read()
    
    start = 0
    end = offsets[0][0]
    label = 'O'
    
    ann_chunks = [(full_text[:end], label)]
    
    for i in range(len(offsets)):
        start = offsets[i][0]
        label = offsets[i][1]
        
        if i < len(offsets) - 1:
            end = offsets[i+1][0]
            term = [(full_text[start:end], label)]
            if term[0]:
                ann_chunks.extend(term)
        
        else:
            term = [(full_text[start:], label)]  
            ann_chunks.extend(term)
    
    return ann_chunks

In [102]:
def bio_labeler(chunks):
    """Helper function that further processes annotated chunks from ann_chunker()
       Tokenizes the chunks and applies BIO labels to each token
       
       Inputs:
       chunks = list of annotated chunks based on .ann file offsets
                format: (chunk, label)
       
       Output:
       bio_doc = document transformed into a list of tokens with bio labels"""
    
    bio_tokens = []
    
    for tup in chunks:
        chunk, label = tup
        
        if label == 'O':
            if chunk[:1] == '\n':
                bio_tokens.append([])
            sentences = sent_tokenize(chunk.strip())
            if sentences:
                for s in sentences:
                    for x in word_tokenize(s):
                        bio_tokens.append([x, label])
                        if x == '.':
                            bio_tokens.append([])
                
        else:
            tokens = [x for x in word_tokenize(chunk)]
            for i in range(len(tokens)):
                if i == 0:
                    bio_tokens.append([tokens[i], 'B-' + label])
                else:
                    bio_tokens.append([tokens[i], 'I-' + label])
    
    return bio_tokens

In [105]:
def generate_ner_files(filepaths, output_path):
    """Helper function that reads .txt and corresponding .ann files from a path
       and generates csv file in CoNLL 2003 format (for use in NER task)
       
       Inputs:
       filepaths = filepaths (folder + filename, but no extension) for .txt and .ann files
       output_path = filepath (folder + filename, but no extension) for output file"""
    
    start = time()
    
    docs = []
    corrections = []
    
    for file in filepaths:
        
        docs.append(['-DOCSTART- -X- O O'])
        corrections.append([f'SENTENCE: {file[-4:]}'])
        
        cleaned_offsets, file_corrections = process_ann(f'{file}.ann')
        corrections.append(file_corrections)
        
        chunks = ann_chunker(f'{file}.txt', cleaned_offsets)
        bio_tokens = bio_labeler(chunks)
        docs.extend(bio_tokens)
    
    with open(f'{output_path}.csv', 'w') as f:
        writer = csv.writer(f, delimiter='\t')
        writer.writerows(docs)
    
    with open(f'{output_path}_corrections.csv', 'w') as f:
        writer = csv.writer(f)
        writer.writerows(corrections)
    
    end = time() - start
    print(f'Finished in {end:.3f} seconds')

#### Test the Helper Functions

In [109]:
test_path = 'raw_data/sample_ee/0001'

In [110]:
with io.open(f'{test_path}.txt', 'r', encoding='utf-8', errors='ignore') as text:
    full_text = text.read()
full_text

'16.8: [5-(2,3-difluorophenyl)-3-methyl-2,4-dioxo-3,4-dihydro-2H-pyrimidin-1-yl]-acetic acid\n2.3 ml (2.3 mmol) of a 1N aqueous solution of lithium hydroxide is added to a solution of 475 mg (1.5 mmol) of [5-(2,3-difluorophenyl)-3-methyl-2,4-dioxo-3,4-dihydro-2H-pyrimidin-1-yl]-methyl acetate in 15 ml of tetrahydrofuran and 3 mL of water. The reaction mixture is stirred at room temperature for 2 hours, and then adjusted to pH6 by adding 4 ml of 1N aqueous solution of acetic acid. The product is extracted with ethyl acetate. The organic phase is washed with water and then with a saturated aqueous solution of sodium chloride, dried over magnesium sulphate, filtered and concentrated under vacuum. 400 mg (88%) of [5-(2,3-difluorophenyl)-3-methyl-2,4-dioxo-3,4-dihydro-2H-pyrimidin-1-yl]-acetic acid is obtained in the form of a white solid.'

In [111]:
with io.open(f'{test_path}.ann', 'r', encoding='utf-8', errors='ignore') as text:
    ann = [x.strip().split('\t') for x in text.readlines()] #if x.strip().split('\t')[0][0] == 'T']
ann[:3]

[['T0', 'REAGENT_CATALYST 332 337', 'water'],
 ['T19', 'WORKUP 549 555', 'washed'],
 ['T1', 'OTHER_COMPOUND 450 457', 'aqueous']]

In [112]:
test_offsets, test_corrections = process_ann(f'{test_path}.ann')

In [113]:
test_offsets[:3]

[(0, 'EXAMPLE_LABEL', 'T10'), (4, 'O', 'X'), (6, 'REACTION_PRODUCT', 'T13')]

In [114]:
test_corrections

{}

In [115]:
trial_sentence = ann_chunker(f'{test_path}.txt', test_offsets)

In [116]:
trial_sentence[:3]

[('', 'O'), ('16.8', 'EXAMPLE_LABEL'), (': ', 'O')]

In [117]:
trial_doc = bio_labeler(trial_sentence)
trial_doc[:3]

[['16.8', 'B-EXAMPLE_LABEL'], [':', 'O'], ['[', 'B-REACTION_PRODUCT']]

In [120]:
generate_ner_files([test_path], 'raw_data/test')
with io.open('raw_data/test.csv', 'r', encoding='utf-8', errors='ignore') as sample:
    output = sample.readlines()
output

Finished in 0.079 seconds


['-DOCSTART- -X- O O\n',
 '16.8\tB-EXAMPLE_LABEL\n',
 ':\tO\n',
 '[\tB-REACTION_PRODUCT\n',
 '5-\tI-REACTION_PRODUCT\n',
 '(\tI-REACTION_PRODUCT\n',
 '2,3-difluorophenyl\tI-REACTION_PRODUCT\n',
 ')\tI-REACTION_PRODUCT\n',
 '-3-methyl-2,4-dioxo-3,4-dihydro-2H-pyrimidin-1-yl\tI-REACTION_PRODUCT\n',
 ']\tI-REACTION_PRODUCT\n',
 '-acetic\tI-REACTION_PRODUCT\n',
 'acid\tI-REACTION_PRODUCT\n',
 '\n',
 '2.3\tO\n',
 'ml\tO\n',
 '(\tO\n',
 '2.3\tO\n',
 'mmol\tO\n',
 ')\tO\n',
 'of\tO\n',
 'a\tO\n',
 '1N\tO\n',
 'aqueous\tB-REAGENT_CATALYST\n',
 'solution\tO\n',
 'of\tO\n',
 'lithium\tB-REAGENT_CATALYST\n',
 'hydroxide\tI-REAGENT_CATALYST\n',
 'is\tO\n',
 'added\tB-REACTION_STEP\n',
 'to\tO\n',
 'a\tO\n',
 'solution\tO\n',
 'of\tO\n',
 '475\tO\n',
 'mg\tO\n',
 '(\tO\n',
 '1.5\tO\n',
 'mmol\tO\n',
 ')\tO\n',
 'of\tO\n',
 '[\tB-STARTING_MATERIAL\n',
 '5-\tI-STARTING_MATERIAL\n',
 '(\tI-STARTING_MATERIAL\n',
 '2,3-difluorophenyl\tI-STARTING_MATERIAL\n',
 ')\tI-STARTING_MATERIAL\n',
 '-3-methyl-2,4-

In [123]:
output[0][:10]

'-DOCSTART-'

#### Process the Raw Data

In [125]:
# generate sample set
path_sample = 'raw_data/sample_ee'
filenames_sample = list({x[:4] for x in os.listdir(path_sample) if x[0] != '.'})
filepath_sample = [f'{path_sample}/{x}' for x in filenames_sample]

output_sample = 'raw_data/sample_data'
generate_ner_files(filepath_sample, output_sample)

with io.open(f'{output_sample}.csv', 'r', encoding='utf-8', errors='ignore') as sample:
    output = sample.readlines()
check = [1 for x in output if x[:10] == '-DOCSTART-']
print(f'Number of train sentences: {len(check)}')

Finished in 0.923 seconds
Number of train sentences: 50


In [126]:
# generate filename list for train, dev, and test sets
path_train = 'raw_data/EE/ee_train'
filenames_train = list({x[:4] for x in os.listdir(path_train) if x[0] != '.'})
print(f'Number of train files: {len(filenames_train)}')

path_dev = 'raw_data/EE/ee_dev'
filenames_dev = list({x[:4] for x in os.listdir(path_dev) if x[0] != '.'})
print(f'Number of dev files: {len(filenames_dev)}')

path_test = 'raw_data/EE/ee_test'
filenames_test = list({x[:4] for x in os.listdir(path_test) if x[0] != '.'})
print(f'Number of test files: {len(filenames_test)}')

path_test_ann = 'raw_data/EE/ee_test_ann'
filenames_test_ann = list({x[:4] for x in os.listdir(path_test_ann) if x[0] != '.'})
print(f'Number of test .ann files: {len(filenames_test_ann)}')

Number of train files: 900
Number of dev files: 225
Number of test files: 9999
Number of test .ann files: 375


In [127]:
# check how many test .txt files match the .ann files
intersect = list(set(filenames_test) & set(filenames_test_ann))
len(intersect)

375

In [132]:
# generate train set
filepath_train = [f'{path_train}/{x}' for x in filenames_train]

output_train = 'data/conll/ee_ner_train'
generate_ner_files(filepath_train, output_train)

with io.open(f'{output_train}.csv', 'r', encoding='utf-8', errors='ignore') as sample:
    output = sample.readlines()
check = [1 for x in output if x[:10] == '-DOCSTART-']
print(f'Number of train sentences: {len(check)}')

Finished in 8.437 seconds
Number of train sentences: 900


In [133]:
# generate dev set
filepath_dev = [f'{path_dev}/{x}' for x in filenames_dev]

output_dev = 'data/conll/ee_ner_dev'
generate_ner_files(filepath_dev, output_dev)

with io.open(f'{output_dev}.csv', 'r', encoding='utf-8', errors='ignore') as sample:
    output = sample.readlines()
check = [1 for x in output if x[:10] == '-DOCSTART-']
print(f'Number of train sentences: {len(check)}')

Finished in 2.932 seconds
Number of train sentences: 225


In [134]:
# generate test set
filepath_test = [f'{path_test}/{x}' for x in intersect]

output_test = 'data/conll/ee_ner_test'
generate_ner_files(filepath_test, output_test)

with io.open(f'{output_test}.csv', 'r', encoding='utf-8', errors='ignore') as sample:
    output = sample.readlines()
check = [1 for x in output if x[:10] == '-DOCSTART-']
print(f'Number of train sentences: {len(check)}')

Finished in 7.688 seconds
Number of train sentences: 375
